# QTM350 - Group Project
## Group members

Please find the names of the group members below:

In [1]:
# Import packages
import sqlite3
import pandas as pd
import random

In [2]:
# Connect to the database
db_file = 'group-project.db'
connection = sqlite3.connect(db_file)
cursor = connection.cursor()

In [3]:
# Execute the SQL commands to create the table
cursor.execute('''
DROP TABLE IF EXISTS groups;
''')

cursor.execute('''
CREATE TABLE groups (
    student_id INTEGER PRIMARY KEY AUTOINCREMENT,
    student_name TEXT NOT NULL UNIQUE,
    emory_id INT UNIQUE, -- Allow NULL and ensure uniqueness
    group_id INT
);
''')

# Commit the changes for table creation
connection.commit()

In [4]:
# Insert data into the table
insert_sql = '''
INSERT INTO groups (student_name, emory_id, group_id) VALUES (?, ?, ?)
'''

# List of tuples: (student_name, emory_id, group_id)
# Danilo Freire removed
# Using Python None here, as executemany handles it correctly for sqlite3
students_data = [
    ('Jacky An', 2486597, 1),
    ('Anushka Basu', 2551669, None),
    ('Manny Bettencourt', 2515917, None),
    ('Kush Bhatia', 2492303, None),
    ('Davis Boor', 2556176, None),
    ('Joyce Chen', 2485906, None),
    ('Nicole Chen', 2608628, None),
    ('Yunqing Ivy Chen', 2585468, None),
    ('Anna Choi', 2452634, 2),
    ('Daisy Cossio', 2503179, None),
    ('Jackson Fang', 2432239, None),
    ('Lucas Goldfein', 2486286, None),
    ('Felix Huang', 2517419, None),
    ('Sean Jeon', 2550044, 2),
    ('Emma Jinright', 2488724, None),
    ('Minjoo Kim', 2580377, None),
    ('Jooha Lee', 2433418, 2),
    ('Krystal Li', 2587957, None),
    ('Zihan Liang', 2609381, None),
    ('Amber Lin', 2486031, None),
    ('Lucas Lobo', 2555247, None),
    ('Katherine Martini', 2549333, None),
    ('Liane Muir', 2493825, None),
    ('Nora Ni', 2599639, None),
    ('Daniel Nickas', 2549711, None),
    ('Kei Nie', 2541685, None),
    ('Phoebe Pan', 2630423, None),
    ('Adam Pastor', 2565464, None),
    ('Aryan Patel', 2494029, None),
    ('Nick Richards', 2495699, None),
    ('Caleb Sharkey', 2514723, None),
    ('Iliyan Sherali', 2520396, None),
    ('Saanvi Sood', 2494531, None),
    ('Brian Tekadtuera', 2577545, None),
    ('Mingke Tian', 2636361, None),
    ('Rahul Ulman', 2434956, None),
    ('Aanya Vusirikala', 2558223, None),
    ('Xinyi Wang', 2549813, None),
    ('Iris Wu', 2515186, None),
    ('Bernice Yuan', 2487091, 1),
    ('Eric Zou', 2548644, 1)
]

cursor.executemany(insert_sql, students_data)

# Commit the transaction for data insertion
connection.commit()

In [ ]:
# --- Random Group Assignment Logic ---
print("\n--- Assigning Remaining Students to Groups ---")

# Ensure connection is still open or reconnect
try:
    # Check if cursor is usable by executing a simple query
    cursor.execute("SELECT 1") 
except (sqlite3.ProgrammingError, AttributeError, NameError): 
    print("Reconnecting to database for group assignment...")
    connection = sqlite3.connect(db_file)
    cursor = connection.cursor()

# Fetch unassigned students (student_id)
cursor.execute("SELECT student_id FROM groups WHERE group_id IS NULL")
unassigned_rows = cursor.fetchall()
unassigned_ids = [row[0] for row in unassigned_rows] # Extract IDs
n = len(unassigned_ids)

print(f"Found {n} unassigned students.")

g4 = 0 # Number of groups of 4
g3 = 0 # Number of groups of 3
possible = True

if n >= 3:
    # Calculate number of groups of 4 and 3, prioritizing groups of 4
    temp_g4 = n // 4
    remainder = n % 4 # Remainder after forming groups of 4

    # Adjust to make remainder divisible by 3
    while remainder % 3 != 0:
        if temp_g4 == 0: # Cannot form any more groups of 4
            possible = False
            print(f"Cannot divide {n} students perfectly into groups of 3 or 4.")
            break
        temp_g4 -= 1    # Reduce one group of 4
        remainder += 4  # Add those 4 students back to the remainder pool

    if possible:
        g4 = temp_g4
        g3 = remainder // 3
        print(f"Creating {g4} groups of 4 and {g3} groups of 3.")

        # Shuffle the student IDs randomly
        random.shuffle(unassigned_ids)

        # Find the next available group ID
        cursor.execute("SELECT MAX(group_id) FROM groups WHERE group_id IS NOT NULL")
        max_existing_group_id_result = cursor.fetchone()
        # Handle case where there are no existing groups (max_existing_group_id_result[0] would be None)
        max_existing_group_id = max_existing_group_id_result[0] if max_existing_group_id_result and max_existing_group_id_result[0] is not None else 0
        next_group_id = max_existing_group_id + 1 
        print(f"Starting new groups from ID: {next_group_id}")

        current_student_index = 0
        update_sql = "UPDATE groups SET group_id = ? WHERE student_id = ?"

        # Assign groups of 4
        print("\nAssigning groups of 4...")
        for i in range(g4):
            group_members_ids = unassigned_ids[current_student_index : current_student_index + 4]
            if group_members_ids: # Ensure there are members to assign
                print(f"  Assigning group {next_group_id} to students (IDs): {group_members_ids}")
                for student_id in group_members_ids:
                    cursor.execute(update_sql, (next_group_id, student_id))
                next_group_id += 1
                current_student_index += 4
            else:
                print("  Warning: Tried to assign an empty group of 4.")

        # Assign groups of 3
        print("\nAssigning groups of 3...")
        for i in range(g3):
            group_members_ids = unassigned_ids[current_student_index : current_student_index + 3]
            if group_members_ids: # Ensure there are members to assign
                print(f"  Assigning group {next_group_id} to students (IDs): {group_members_ids}")
                for student_id in group_members_ids:
                    cursor.execute(update_sql, (next_group_id, student_id))
                next_group_id += 1
                current_student_index += 3
            else:
                 print("  Warning: Tried to assign an empty group of 3.")

        # Commit the updates for group assignments
        connection.commit()
        print("\nFinished assigning groups.")

else:
    print("Not enough unassigned students (need at least 3) to form new groups.")

# The final query cell will display the results.
print("--- End of Group Assignment ---\n")


--- Assigning Remaining Students to Groups ---
Found 35 unassigned students.
Creating 8 groups of 4 and 1 groups of 3.
Starting new groups from ID: 3

Assigning groups of 4...
  Assigning group 3 to students (IDs): [30, 31, 22, 34]
  Assigning group 4 to students (IDs): [11, 18, 2, 4]
  Assigning group 5 to students (IDs): [38, 13, 7, 15]
  Assigning group 6 to students (IDs): [19, 8, 33, 16]
  Assigning group 7 to students (IDs): [21, 32, 28, 29]
  Assigning group 8 to students (IDs): [37, 36, 23, 20]
  Assigning group 9 to students (IDs): [24, 10, 5, 27]
  Assigning group 10 to students (IDs): [6, 12, 3, 25]

Assigning groups of 3...
  Assigning group 11 to students (IDs): [39, 26, 35]

Finished assigning groups.
--- End of Group Assignment ---



In [6]:
# Query the table and display results using pandas
query = '''
SELECT * FROM groups
ORDER BY group_id, student_name
'''
# Reconnect if connection was closed, or ensure it's open
connection = sqlite3.connect(db_file)
df = pd.read_sql_query(query, connection)

# Close the connection
connection.close()

# Display the DataFrame
df

,student_id,student_name,emory_id,group_id
0,40,Bernice Yuan,2487091,1
1,41,Eric Zou,2548644,1
2,1,Jacky An,2486597,1
3,9,Anna Choi,2452634,2
4,17,Jooha Lee,2433418,2
5,14,Sean Jeon,2550044,2
6,34,Brian Tekadtuera,2577545,3
7,31,Caleb Sharkey,2514723,3
8,22,Katherine Martini,2549333,3
9,30,Nick Richards,2495699,3
